# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from glob import glob

from pyopia.classify import Classify
import exampledata
import pyopia.io
from pyopia.pipeline import Pipeline
from pyopia.instrument.silcam import SilCamLoad, ImagePrep, silcam_steps
import pyopia.process

In [3]:
filename = exampledata.get_example_silc_image()

files = glob('raw_data/*.silc')

In [4]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

datafile_hdf = 'proc/test'
model_path = exampledata.get_example_model()
threshold = 0.85

steps = {'classifier': Classify(model_path=model_path),
         'load': SilCamLoad(),
         'imageprep': ImagePrep(),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)}

steps_for_silcam, initial_steps = silcam_steps(model_path, threshold, datafile_hdf)

print(steps_for_silcam)

{'classifier': <pyopia.classify.Classify object at 0x000001FE11979D60>, 'load': <pyopia.instrument.silcam.SilCamLoad object at 0x000001FE1A480040>, 'imageprep': <pyopia.instrument.silcam.ImagePrep object at 0x000001FE1A4802B0>, 'segmentation': <pyopia.process.Segment object at 0x000001FE183196A0>, 'statextract': <pyopia.process.CalculateStats object at 0x000001FE19BAB040>, 'output': <pyopia.io.StatsH5 object at 0x000001FE18B09F70>}


In [5]:
processing_pipeline = Pipeline(steps_for_silcam, initial_steps=initial_steps)
for filename in files[:2]:
    stats = processing_pipeline.run(filename)

Initialising pipeline
  Running <pyopia.classify.Classify object at 0x000001FE11979D60>
Pipeline ready with these data:  ['cl']


In [6]:
# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

OSError: Unable to open file (unable to open file: name = 'proc/test-STATS.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import pyopia.statistics
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))
print('Number of raw images: ', pyopia.statistics.count_images_in_stats(stats))